In [2]:
import pandas as pd
import numpy as np

In [3]:
rating = pd.read_csv("ratings.csv")
ratings_200 =  rating['userId'] <= 200 #get first 200 users
ratings = rating[ratings_200]
print(ratings.shape)

(29268, 4)


In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
#importing movie title and joining with main data
movies=pd.read_csv("movies.csv")[0:200] #use 200 movies only
movie_titles= movies.drop('genres', 1)
df=pd.merge(ratings, movie_titles, on='movieId')
df.head()
#movie_titles.head()

,userId,movieId,rating,timestamp,title
0,1,1,4.0,964982703,Toy Story (1995)
1,5,1,4.0,847434962,Toy Story (1995)
2,7,1,4.5,1106635946,Toy Story (1995)
3,15,1,2.5,1510577970,Toy Story (1995)
4,17,1,4.5,1305696483,Toy Story (1995)


In [6]:
# creating mean ratings data
data=pd.DataFrame(df.groupby('title')['rating'].mean())
data.head()

,rating
title,
Ace Ventura: When Nature Calls (1995),2.517857
"Amazing Panda Adventure, The (1995)",3.333333
"American President, The (1995)",3.851852
Angels and Insects (1995),4.000000
Anne Frank Remembered (1995),5.000000


In [7]:
#creating number of ratings data
data['number_of_ratings'] = df.groupby('title')['rating'].count()
data.head(200)

,rating,number_of_ratings
title,,
Ace Ventura: When Nature Calls (1995),2.517857,28
"Amazing Panda Adventure, The (1995)",3.333333,3
"American President, The (1995)",3.851852,27
Angels and Insects (1995),4.000000,4
Anne Frank Remembered (1995),5.000000,1
...,...,...
"Walk in the Clouds, A (1995)",3.500000,8
Waterworld (1995),2.888889,36
White Man's Burden (1995),3.000000,1


In [15]:
# use this "fill_value" to convert "NaN" value to 0 and "aggfunc wth lambda ( anonymous function)" to convert any integer(rating) into 1 so that cosine function can be implemented
user_movie_rating = df.pivot_table(index='userId', columns='title', values='rating',aggfunc=lambda x: len(x.unique()),fill_value=0)
user_movie_rating.head(200)

title,Ace Ventura: When Nature Calls (1995),"Amazing Panda Adventure, The (1995)","American President, The (1995)",Angels and Insects (1995),Anne Frank Remembered (1995),Antonia's Line (Antonia) (1995),Apollo 13 (1995),Assassins (1995),"Awfully Big Adventure, An (1995)",Babe (1995),...,Unstrung Heroes (1995),Up Close and Personal (1996),"Usual Suspects, The (1995)",Vampire in Brooklyn (1995),Waiting to Exhale (1995),"Walk in the Clouds, A (1995)",Waterworld (1995),White Man's Burden (1995),White Squall (1996),Wild Bill (1995)
userId,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,0,0,1,...,0,0,1,0,0,0,0,0,0,0
6,1,1,1,0,0,0,1,0,0,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
197,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
198,0,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0


In [56]:
#Create a temporary dataframe pivot item vs. item
tmpMovieRatings = pd.DataFrame(index=user_movie_rating.columns,columns=user_movie_rating.columns)
#then calculate item based cosine similarity for all items and store it in this temporary dataframe pivot
for i in range(0,len(tmpMovieRatings.columns)) :
    for j in range(0,len(tmpMovieRatings.columns)) : # cosine similarity = 1 - cosine function
        tmpMovieRatings.iloc[i,j] = 1-cosine(user_movie_rating.iloc[:,i],user_movie_rating.iloc[:,j])
        
tmpMovieRatings.head()

title,Ace Ventura: When Nature Calls (1995),"Amazing Panda Adventure, The (1995)","American President, The (1995)",Angels and Insects (1995),Anne Frank Remembered (1995),Antonia's Line (Antonia) (1995),Apollo 13 (1995),Assassins (1995),"Awfully Big Adventure, An (1995)",Babe (1995),...,Unstrung Heroes (1995),Up Close and Personal (1996),"Usual Suspects, The (1995)",Vampire in Brooklyn (1995),Waiting to Exhale (1995),"Walk in the Clouds, A (1995)",Waterworld (1995),White Man's Burden (1995),White Squall (1996),Wild Bill (1995)
title,,,,,,,,,,,,,,,,,,,,,
Ace Ventura: When Nature Calls (1995),1,0.218218,0.400066,0,0,0.109109,0.442374,0.0714286,0,0.379088,...,0.218218,0.154303,0.328165,0.188982,0.188982,0.267261,0.409462,0.188982,0.154303,0.133631
"Amazing Panda Adventure, The (1995)",0.218218,1,0.222222,0,0,0.333333,0.202721,0.218218,0,0.267261,...,0.666667,0.235702,0.143223,0.288675,0.288675,0.204124,0.288675,0.57735,0.471405,0.816497
"American President, The (1995)",0.400066,0.222222,1,0.096225,0,0,0.495541,0.145479,0,0.41574,...,0.111111,0.31427,0.405798,0.096225,0.19245,0.204124,0.352825,0.19245,0.31427,0.272166
Angels and Insects (1995),0,0,0.096225,1,0.5,0.288675,0.175562,0,0.5,0.154303,...,0,0.204124,0.186052,0.25,0,0,0.0833333,0,0,0
Anne Frank Remembered (1995),0,0,0,0.5,1,0.57735,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
# get Toy Story (1995) and 'Waiting to Exhale (1995)' movies from tmpMovieRatings and then marge them in one dataframe
tmpMovieRatings.loc['Waiting to Exhale (1995)']
tmpMovieRatings.loc['Toy Story (1995)']
similarity_movies=pd.merge(tmpMovieRatings.loc['Toy Story (1995)'],tmpMovieRatings.loc['Waiting to Exhale (1995)'],on = 'title')
similarity_movies


,Toy Story (1995),Waiting to Exhale (1995)
title,,
Ace Ventura: When Nature Calls (1995),0.381277,0.188982
"Amazing Panda Adventure, The (1995)",0.137038,0.288675
"American President, The (1995)",0.274075,0.19245
Angels and Insects (1995),0.178017,0
Anne Frank Remembered (1995),0.118678,0
...,...,...
"Walk in the Clouds, A (1995)",0.125877,0.353553
Waterworld (1995),0.435153,0.0833333
White Man's Burden (1995),0,0.5


In [60]:
#get the top 10 related movies for Toy Story (1995)
movies_like_toy_story=pd.DataFrame(similarity_movies.iloc[:,0].sort_values(ascending = False).head(11)).rename(columns = {'Toy Story (1995)': 'cosine similarity'})
movies_like_toy_story

,cosine similarity
title,
Toy Story (1995),1
Braveheart (1995),0.514272
Apollo 13 (1995),0.500048
Happy Gilmore (1996),0.49025
Seven (a.k.a. Se7en) (1995),0.444487
"Usual Suspects, The (1995)",0.441606
Babe (1995),0.439499
Batman Forever (1995),0.436051
Waterworld (1995),0.435153


In [61]:
#get the top 10 related movies for Waiting to Exhale (1995)
movies_like_Waiting_to_Exhale=pd.DataFrame(similarity_movies.iloc[:,1].sort_values(ascending = False).head(11)).rename(columns = {'Waiting to Exhale (1995)': 'cosine similarity'})
movies_like_Waiting_to_Exhale

,cosine similarity
title,
Waiting to Exhale (1995),1
Before and After (1996),0.57735
Reckless (1995),0.5
Three Wishes (1995),0.5
Georgia (1995),0.5
Mad Love (1995),0.5
Now and Then (1995),0.5
"Cure, The (1995)",0.5
Bushwhacked (1995),0.5


In [62]:
#recommend 3 films to userID 200 based on number of rating
user_movie_rating = df.pivot_table(index='userId', columns='title', values='rating')
pd.DataFrame(user_movie_rating.loc[200].sort_values(ascending = False).head(3)).rename(columns = {200: 'recommend 3 films to userID 200'})

,recommend 3 films to userID 200
title,
Clueless (1995),5.0
Happy Gilmore (1996),4.5
GoldenEye (1995),4.5
